# Solving the SLIM MIP

This script shows how to solve the SLIM MIP using CPLEX. Here, we solve the MIP, run unit tests on the solution, and output the resulting scoring system model as well as some statistics about the MIP solver. Edit the variables in the following cell in order to pick a different instance.

In [4]:
data_name = 'breastcancer'
instance_name = data_name + '_max_5_features'
repository_dir = '/Users/berk/Desktop/Dropbox (MIT)/Research/SLIM/Toolboxes/miplib2017-slim'
data_file = repository_dir + '/models/data/' + data_name + '_processed.csv'
instance_file = repository_dir + '/instances/' + instance_name + '.mps'
instance_info_file = repository_dir + '/misc/' + instance_name + '.p'

The next cell loads packages and files needed to run the script. Use ``slim_mip.parameters`` to pass parameters for CPLEX.

In [12]:
import os
import sys
import numpy as np
import cplex as cpx
import pickle
import slim as slim
from pprint import pprint

#load IP
slim_mip = cpx.Cplex(instance_file)
slim_info = pickle.load(open(instance_info_file))
data = slim.load_data_from_csv(data_file)

#set CPLEX IP parameters
slim_mip.parameters.timelimit.set(60)
slim_mip.parameters.randomseed.set(0)
slim_mip.parameters.output.clonelog.set(0)
slim_mip.parameters.threads.set(1)
slim_mip.parameters.parallel.set(1)
slim_mip.parameters.mip.tolerances.mipgap.set(np.finfo(np.float).eps)
slim_mip.parameters.mip.tolerances.absmipgap.set(np.finfo(np.float).eps)
slim_mip.parameters.mip.tolerances.integrality.set(np.finfo(np.float).eps)


Selected objective sense:  MINIMIZE
Selected objective  name:  obj
Selected RHS        name:  rhs
Selected bound      name:  bnd


We now solve the slim_mip

In [13]:
slim_mip.solve()

Found incumbent of value 228.271421 after 0.00 sec. (0.28 ticks)
Tried aggregator 2 times.
MIP Presolve modified 683 coefficients.
Aggregator did 3 substitutions.
Reduced MIP has 720 rows, 712 columns, and 7595 nonzeros.
Reduced MIP has 692 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.39 ticks)
Probing time = 0.00 sec. (0.89 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 234 rows and 0 columns.
Reduced MIP has 486 rows, 478 columns, and 5021 nonzeros.
Reduced MIP has 458 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (2.18 ticks)
Probing time = 0.00 sec. (0.27 ticks)
Tried aggregator 1 time.
Reduced MIP has 486 rows, 478 columns, and 5021 nonzeros.
Reduced MIP has 458 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.56 ticks)
Probing time = 0.00 sec. (0.27 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root 

Once we have solved the mip, we can use:

- ``slim.check_ip_solution`` to make sure that the solution passes unit tests
- ``slim.get_slim_summary(slim_mip, slim_info, data)`` to get information about the MIP and SLIM scoring system.

In [14]:
slim.check_slim_ip_solution(slim_mip, slim_info, data)
slim_results = slim.get_slim_summary(slim_mip, slim_info, data)
pprint(slim_results)

{'L0_norm': 13.0,
 'error_rate': 0,
 'false_negatives': 2,
 'false_positive_rate': 0,
 'false_positives': 12,
 'mistakes': 14,
 'nodes_processed': 57656L,
 'nodes_remaining': 26402L,
 'objective_value': 14.453078947368418,
 'objval_lowerbound': 0.28406769853810926,
 'optimality_gap': 10.347426073971466,
 'pretty_model': <prettytable.PrettyTable object at 0x10c247810>,
 'rho': array([-46.,   4.,   2.,   2.,   0.,   0.,   3.,   0.,   2.,   0.]),
 'simplex_iterations': 1723965L,
 'solution_status': 'time limit exceeded',
 'solution_status_code': 107,
 'string_model': '',
 'true_negatives': 432,
 'true_positive_rate': 0,
 'true_positives': 237}


The ``slim_results`` object also contains a nice printout of the SLIM scoring system. 

In [16]:
print(slim_results['pretty_model'])

+-------------------------------+------------------+-----------+
| PREDICT B IF SCORE >= -46     |                  |           |
| ============================= | ================ | ========= |
| ClumpThickness                |         4 points |   + ..... |
| BareNuclei                    |         3 points |   + ..... |
| UniformityOfCellSize          |         2 points |   + ..... |
| UniformityOfCellShape         |         2 points |   + ..... |
| NormalNucleoli                |         2 points |   + ..... |
| ============================= | ================ | ========= |
| ADD POINTS FROM ROWS 1 to 5   |            SCORE |   = ..... |
+-------------------------------+------------------+-----------+
